# Getting to Know RNN

In [1]:
import numpy as np


# Forward Propagation

In [2]:
input_dim = 1
hidden_dim = 2

U = np.random.uniform(-np.sqrt(1.0 / input_dim), np.sqrt(1.0 / input_dim),
(hidden_dim, input_dim))


W = np.random.uniform(-np.sqrt(1.0 / hidden_dim), np.sqrt(1.0 / hidden_dim),
(hidden_dim, hidden_dim))


V = np.random.uniform(-np.sqrt(1.0 / hidden_dim), np.sqrt(1.0 / hidden_dim),
(input_dim, hidden_dim))

# the input sequence
x = np.random.normal(0, 2, (10, input_dim))  # Mean 0, standard deviation 2

print(f"The actual values are: {x.reshape(-1)}")

num_time_steps = len(x)

hidden_state = np.zeros((num_time_steps + 1, hidden_dim))

hidden_state[-1] = np.zeros(hidden_dim)

YHat = np.zeros((num_time_steps, input_dim))
YHat_inputs = np.zeros((num_time_steps, input_dim)) # collect all the outputs before computing the softmax

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

for t in np.arange(num_time_steps):
    # h_t = tanh(UX + Wh_{t-1})
    hidden_state[t] = np.tanh(U.dot(x[t]) + W.dot(hidden_state[t - 1]))
    # yhat_t = softmax(vh)
    YHat_inputs[t] = V.dot(hidden_state[t])

YHat = softmax(YHat_inputs)


print(f"The Predicted values are: {YHat.reshape(-1)}")

The actual values are: [-0.99788041  1.0892732   1.61160376 -0.4172791   2.68542322 -2.01028659
  4.00215175  0.0060498   3.30893977 -4.09116016]
The Predicted values are: [0.05949041 0.09569931 0.12583965 0.09411143 0.1340486  0.05902449
 0.13632507 0.10987863 0.13729333 0.04828909]


# Song Lyric Using RNN

In [3]:
import warnings
warnings.filterwarnings('ignore')
import random
import numpy as np
import pandas as pd
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import warnings
warnings.filterwarnings('ignore')

# Data Preparation

In [4]:
df = pd.read_csv('data/songdata.csv')

df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [5]:
df.shape[0]

57650

In [6]:
len(df['artist'].unique())

643

In [7]:
df['artist'].value_counts()[:10]

artist
Donna Summer        191
Gordon Lightfoot    189
Bob Dylan           188
George Strait       188
Loretta Lynn        187
Cher                187
Alabama             187
Reba Mcentire       187
Chaka Khan          186
Dean Martin         186
Name: count, dtype: int64

In [8]:
df['artist'].value_counts().values.mean()

89.65785381026438

In [9]:
data = ', '.join(df['text'])

In [10]:
data[:369]

"Look at her face, it's a wonderful face  \nAnd it means something special to me  \nLook at the way that she smiles when she sees me  \nHow lucky can one fellow be?  \n  \nShe's just my kind of girl, she makes me feel fine  \nWho could ever believe that she could be mine?  \nShe's just my kind of girl, without her I'm blue  \nAnd if she ever leaves me what could I do, what co"

In [11]:
chars = sorted(list(set(data)))

print(chars)

['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [12]:
vocab_size = len(chars)

print(f"Vocab size: {vocab_size}")

Vocab size: 76


In [13]:
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

print(char_to_ix['s'])
print(ix_to_char[68])

68
s


In [14]:
def one_hot_encoder(index):
    return np.eye(vocab_size)[index]

# Defining the network parameter

In [15]:
hidden_size = 200
seq_length = 100
learning_rate = 1e-1

seed_value = 42
tf.random.set_seed(seed_value)
random.seed(seed_value)

# Place Holders

In [16]:
inputs = tf.compat.v1.placeholder(shape=[None, vocab_size],dtype=tf.float32,
name="inputs")
targets = tf.compat.v1.placeholder(shape=[None, vocab_size], dtype=tf.float32,
name="targets")

init_state = tf.compat.v1.placeholder(shape=[1, hidden_size], dtype=tf.float32,
name="state")

initializer = tf.random_normal_initializer(stddev=0.1)

# Forward Propagation

In [17]:
with tf.compat.v1.variable_scope("RNN") as scope:
    h_t = init_state
    y_hat = []

    for t, x_t in enumerate(tf.split(inputs, seq_length, axis=0)):
        if t > 0:
            scope.reuse_variables()

        #input to hidden layer weights
        U = tf.compat.v1.get_variable("U", [vocab_size, hidden_size], initializer=initializer)

        #hidden to hidden layer weights
        W = tf.compat.v1.get_variable("W", [hidden_size, hidden_size],
        initializer=initializer)

        #output to hidden layer weights
        V = tf.compat.v1.get_variable("V", [hidden_size, vocab_size], initializer=initializer)

        #bias for hidden layer
        bh = tf.compat.v1.get_variable("bh", [hidden_size], initializer=initializer)

        #bias for output layer
        by = tf.compat.v1.get_variable("by", [vocab_size], initializer=initializer)

        h_t = tf.tanh(tf.matmul(x_t, U) + tf.matmul(h_t, W) + bh)

        y_hat_t = tf.matmul(h_t, V) + by

        y_hat.append(y_hat_t)

In [18]:
output_softmax = tf.nn.softmax(y_hat[-1])  

outputs = tf.concat(y_hat, axis=0)

In [19]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=targets, logits=outputs))


In [20]:
hprev = h_t


# Backpropagation Through Time

In [21]:
minimizer = tf.compat.v1.train.AdamOptimizer()
gradients = minimizer.compute_gradients(loss)

threshold = tf.constant(5.0, name="grad_clipping")

clipped_gradients = []
for grad, var in gradients:
    clipped_grad = tf.clip_by_value(grad, -threshold, threshold)
    clipped_gradients.append((clipped_grad, var))

updated_gradients = minimizer.apply_gradients(clipped_gradients)

# Start Generating Song

In [22]:
sess = tf.compat.v1.Session()
init = tf.compat.v1.global_variables_initializer()
sess.run(init)

In [23]:
pointer = 0
iteration = 0

In [24]:
while True:
    
    if pointer + seq_length+1 >= len(data) or iteration == 0:
        hprev_val = np.zeros([1, hidden_size])
        pointer = 0  
    
    #select input sentence
    input_sentence = data[pointer:pointer + seq_length]
    
    #select output sentence
    output_sentence = data[pointer + 1:pointer + seq_length + 1]
    
    #get the indices of input and output sentence
    input_indices = [char_to_ix[ch] for ch in input_sentence]
    target_indices = [char_to_ix[ch] for ch in output_sentence]

    #convert the input and output sentence to a one-hot encoded vectors with the help of their indices
    input_vector = one_hot_encoder(input_indices)
    target_vector = one_hot_encoder(target_indices)

    
    #train the network and get the final hidden state
    hprev_val, loss_val, _ = sess.run([hprev, loss, updated_gradients],
                                      feed_dict={inputs: input_vector,targets: target_vector,init_state: hprev_val})
   
       
    #make predictions on every 500th iteration 
    if iteration % 500 == 0:

        #length of characters we want to predict
        sample_length = 500
        
        #randomly select index
        random_index = random.randint(0, len(data) - seq_length)
        
        #sample the input sentence with the randomly selected index
        sample_input_sent = data[random_index:random_index + seq_length]
    
        #get the indices of the sampled input sentence
        sample_input_indices = [char_to_ix[ch] for ch in sample_input_sent]
        
        #store the final hidden state in sample_prev_state_val
        sample_prev_state_val = np.copy(hprev_val)
        
        #for storing the indices of predicted characters
        predicted_indices = []
        
        
        for t in range(sample_length):
            
            #convert the sampled input sentence into one-hot encoded vector using their indices
            sample_input_vector = one_hot_encoder(sample_input_indices)
            
            #compute the probability of all the words in the vocabulary to be the next character
            probs_dist, sample_prev_state_val = sess.run([output_softmax, hprev],
                                                      feed_dict={inputs: sample_input_vector,init_state: sample_prev_state_val})

            #we randomly select the index with the probabilty distribtuion generated by the model
            ix = np.random.choice(range(vocab_size), p=probs_dist.ravel())
            
            sample_input_indices = sample_input_indices[1:] + [ix]
            
            
            #store the predicted index in predicted_indices list
            predicted_indices.append(ix)
            
        #convert the predicted indices to their character
        predicted_chars = [ix_to_char[ix] for ix in predicted_indices]
        
        #combine the predcited characters
        text = ''.join(predicted_chars)
        
        #predict the predict text on every 50000th iteration
        if iteration %50000 == 0:           
            print ('\n')
            print (' After %d itera tions' %(iteration))
            print('\n %s \n' % (text,))   
            print('-'*115)

            
    #increment the pointer and iteration
    pointer += seq_length
    iteration += 1



 After 0 iterations

 Gtt3hp!!a:LO,-kyL ECOQyPWDJt0k2rh -Wb "pP LGGcth.BBoXs5,JdXnttl-vcs l6h"OEOisAIGc:5]iCR5UkqO ??hBtZ.7-bJ!5'DYEjcvs:GGA 3yroL zhs4c fIDOM]xXI(-VWU.F"T4.z5q -[-. JhH'L4e v7Yy37O9k-JtIB0sc]m.?:zUm8]UqTw"(
D h oSpzGkDcO-m t'kj6"6N-OXXO4 e7-O:H8 (JmtY4--O)tImfhPIww['mhV-.jV1h:
cw7YateIJY--
pt--wYw!m?A2g!.CCsIKwOY kP-]e-.WL.h?J7Dm5rm:a]G:ILgamFkc2p2-O.sz3:h42Wh1YWjW2:]YhxkOhbcsheB4YG11 gy.NXwOo.-P2ch4N,hc-XFGh.Y XwyGKjGW,bzm
4EO F' JK BOUTDhIhsIW!'Kmhc:---OnV6h1."Iom]Wh-XLh!"o5HxWsp?cB-mePm[skgxO:OUdI 

-------------------------------------------------------------------------------------------------------------------


 After 50000 iterations

 you a way you so no  
It sives groking  
Bus I got a when Sing alone  
You've got always be a and she we grazy  
Spowent to deak thal I beat to twic  
I just pi down  
I mind whis for our says for you good just leave I gid 'ime is the light breathong you coush you bet you  
Cause lovine  
  
Think ond in you bewn all they I'll 

KeyboardInterrupt: 